In [ ]:
import numpy as np
import zarr
import cupy as cp
from itertools import product

from decorrelation.shp import ks_test
from decorrelation.co import emperical_co_sp
from decorrelation.pl import emi

In [ ]:
from dask import array as da
from dask import delayed
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster

In [ ]:
cluster = LocalCUDACluster()
client = Client(cluster)
client

2023-04-12 15:46:27,548 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-12 15:46:27,548 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-12 15:46:27,548 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-12 15:46:27,548 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 100.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35802,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 100.00 GiB
Comm: tcp://127.0.0.1:43875,Total threads: 1
Dashboard: http://127.0.0.1:43682/status,Memory: 50.00 GiB
Nanny: tcp://127.0.0.1:33124,


In this tutorial, we demostrate how to use `Dask` for distributed computing.

Two significant issues for InSAR big data processing are: 1) the memory of CPU/GPU does not fit the volume of very big data;
2) the processing speed is limited. For the first issue, one common solution is divide the data into independent chunks
and process the chunks one by one. In many case, the processing of chunks are independent. Thus the processing can be speeded
up by parallel processing.

`Dask` is a job scheduler that allows deploying process-level parallel processing.
Through the `Delayed` feature, `Dask` operations only construct the computing workflow rather than do the computation immediately.
All of the computations are done at the end to allow `Dask` better distributing computing resources for the task.
`Dask` makes the parallel processing easier and enable the decoupling of codes for computation and codes for scheduling.

In this demo, we use `Dask` for multi-GPU KS test. This includes:

- Read SLC data from disk to CPU and then transfer to GPU;
- Divide them into overlapped chunks;
- Apply the `ks_test` to each chunks;
- Trim the overlapping in the results;
- Transfer the results to CPU;
- Save data to disk.

## Processing

Load data into CPU, the chunk size is setted:

In [ ]:
rslc_path = '../../data/rslc.zarr'
rslc_zarr = zarr.open(rslc_path,mode='r')
cpu_rslc = da.from_zarr(rslc_path,chunks=(1000,1000,17))

Then convert to GPU:

In [ ]:
rslc = cpu_rslc.map_blocks(cp.asarray)

In [ ]:
rslc

dask.array<asarray, shape=(2500, 1834, 17), dtype=complex64, chunksize=(1000, 1000, 17), chunktype=cupy.ndarray>

Prepare the `rmli` and processing parameters:

In [ ]:
rmli = da.abs(rslc)**2
rmli

dask.array<pow, shape=(2500, 1834, 17), dtype=float32, chunksize=(1000, 1000, 17), chunktype=cupy.ndarray>

In [ ]:
az_half_win = 5
r_half_win = 5
az_win = 2*az_half_win+1
r_win = 2*r_half_win+1

Enable overlapping between chunks. This is because the KS test is conducted on small region (az_win*r_win).

In [ ]:
depth = {0:az_half_win, 1:r_half_win, 2:0}; boundary = {0:'none',1:'none',2:'none'}
rmli_overlap = da.overlap.overlap(rmli,depth=depth, boundary=boundary)
rmli_overlap

dask.array<overlap, shape=(2520, 1844, 17), dtype=float32, chunksize=(1010, 1005, 17), chunktype=cupy.ndarray>

In [ ]:
# the array of delayed object will lost some shape information so perserve them here
chunks_size = rmli_overlap.chunks 
nchunks = tuple(len(nchunks_in_each_dim) for nchunks_in_each_dim in chunks_size)
chunks_shape = list(product(*chunks_size))

In [ ]:
chunks_size

((1005, 1010, 505), (1005, 839), (17,))

In [ ]:
nchunks

(3, 2, 1)

In [ ]:
chunks_shape

[(1005, 1005, 17),
 (1005, 839, 17),
 (1010, 1005, 17),
 (1010, 839, 17),
 (505, 1005, 17),
 (505, 839, 17)]

Convert each chunk to delayed object;

In [ ]:
rmli_chunks = rmli_overlap.to_delayed()
rmli_chunks

array([[[Delayed(('overlap-deeb6bf02a78ef34f82e8a8fcbd6af40', 0, 0, 0))],
        [Delayed(('overlap-deeb6bf02a78ef34f82e8a8fcbd6af40', 0, 1, 0))]],

       [[Delayed(('overlap-deeb6bf02a78ef34f82e8a8fcbd6af40', 1, 0, 0))],
        [Delayed(('overlap-deeb6bf02a78ef34f82e8a8fcbd6af40', 1, 1, 0))]],

       [[Delayed(('overlap-deeb6bf02a78ef34f82e8a8fcbd6af40', 2, 0, 0))],
        [Delayed(('overlap-deeb6bf02a78ef34f82e8a8fcbd6af40', 2, 1, 0))]]],
      dtype=object)

Apply the KS test:

In [ ]:
delayed_ks_test = delayed(ks_test,pure=True,nout=2)
results = [delayed_ks_test(rmli_chunk,az_half_win=az_half_win,r_half_win=r_half_win) for rmli_chunk in rmli_chunks.ravel()]
dist_chunks, p_chunks = zip(*results)
dist_chunks, p_chunks = np.array(dist_chunks), np.array(p_chunks)

In [ ]:
dist_chunks

array([Delayed('getitem-f50335529b047b79b2c90cdef57d40fd'),
       Delayed('getitem-4ce12862641d4b5c6c4cc84e652dabc2'),
       Delayed('getitem-c0334ded367e1297dcd1fed323c211e8'),
       Delayed('getitem-da7af3e1e8721cc553402893fa56e087'),
       Delayed('getitem-237696b05fba3e9aec0a3bc835be6a4b'),
       Delayed('getitem-cac2e878f9a43c9f1da9233851482b57')], dtype=object)

Convert `Delayed` object to dask array:

In [ ]:
for i in range(len(chunks_shape)):
    dist_chunks[i] = da.from_delayed(dist_chunks[i],shape=(*chunks_shape[i][:-1],az_win,r_win),meta=cp.array((),dtype=rmli.dtype))
    p_chunks[i] = da.from_delayed(p_chunks[i],shape=(*chunks_shape[i][:-1],az_win,r_win),meta=cp.array((),dtype=rmli.dtype))

In [ ]:
dist_chunks

array([dask.array<from-value, shape=(1005, 1005, 11, 11), dtype=float32, chunksize=(1005, 1005, 11, 11), chunktype=cupy.ndarray>,
       dask.array<from-value, shape=(1005, 839, 11, 11), dtype=float32, chunksize=(1005, 839, 11, 11), chunktype=cupy.ndarray>,
       dask.array<from-value, shape=(1010, 1005, 11, 11), dtype=float32, chunksize=(1010, 1005, 11, 11), chunktype=cupy.ndarray>,
       dask.array<from-value, shape=(1010, 839, 11, 11), dtype=float32, chunksize=(1010, 839, 11, 11), chunktype=cupy.ndarray>,
       dask.array<from-value, shape=(505, 1005, 11, 11), dtype=float32, chunksize=(505, 1005, 11, 11), chunktype=cupy.ndarray>,
       dask.array<from-value, shape=(505, 839, 11, 11), dtype=float32, chunksize=(505, 839, 11, 11), chunktype=cupy.ndarray>],
      dtype=object)

Reshape and concentate the dask array:

In [ ]:
dist_chunks, p_chunks = dist_chunks.reshape((*nchunks,1)).tolist(), p_chunks.reshape((*nchunks,1)).tolist()

In [ ]:
dist_chunks

[[[[dask.array<from-value, shape=(1005, 1005, 11, 11), dtype=float32, chunksize=(1005, 1005, 11, 11), chunktype=cupy.ndarray>]],
  [[dask.array<from-value, shape=(1005, 839, 11, 11), dtype=float32, chunksize=(1005, 839, 11, 11), chunktype=cupy.ndarray>]]],
 [[[dask.array<from-value, shape=(1010, 1005, 11, 11), dtype=float32, chunksize=(1010, 1005, 11, 11), chunktype=cupy.ndarray>]],
  [[dask.array<from-value, shape=(1010, 839, 11, 11), dtype=float32, chunksize=(1010, 839, 11, 11), chunktype=cupy.ndarray>]]],
 [[[dask.array<from-value, shape=(505, 1005, 11, 11), dtype=float32, chunksize=(505, 1005, 11, 11), chunktype=cupy.ndarray>]],
  [[dask.array<from-value, shape=(505, 839, 11, 11), dtype=float32, chunksize=(505, 839, 11, 11), chunktype=cupy.ndarray>]]]]

In [ ]:
dist = da.block(dist_chunks)
p = da.block(p_chunks)

In [ ]:
dist

dask.array<concatenate, shape=(2520, 1844, 11, 11), dtype=float32, chunksize=(1010, 1005, 11, 11), chunktype=cupy.ndarray>

In [ ]:
p

dask.array<concatenate, shape=(2520, 1844, 11, 11), dtype=float32, chunksize=(1010, 1005, 11, 11), chunktype=cupy.ndarray>

In [ ]:
dist.chunks

((1005, 1010, 505), (1005, 839), (11,), (11,))

Trim the overlap region:

In [ ]:
depth = {0:az_half_win, 1:r_half_win, 2:0, 3:0}; boundary = {0:'none',1:'none',2:'none',3:'none'}
dist = da.overlap.trim_overlap(dist,depth=depth,boundary=boundary)
p = da.overlap.trim_overlap(p,depth=depth,boundary=boundary)

In [ ]:
dist.chunks

((1000, 1000, 500), (1000, 834), (11,), (11,))

Copy data from GPU to CPU:

In [ ]:
cpu_dist = da.map_blocks(cp.asnumpy,dist)
cpu_p = da.map_blocks(cp.asnumpy,p)

In [ ]:
cpu_dist

dask.array<asnumpy, shape=(2500, 1834, 11, 11), dtype=float32, chunksize=(1000, 1000, 11, 11), chunktype=numpy.ndarray>

Save the data to disk:

In [ ]:
%%time
cpu_dist.to_zarr('dist.zarr',overwrite=True)
cpu_p.to_zarr('p.zarr',overwrite=True)

CPU times: user 909 ms, sys: 885 ms, total: 1.79 s
Wall time: 39.1 s


# Test

In [ ]:
dask_dist = zarr.load('dist.zarr')
dask_p = zarr.load('p.zarr')
dist = zarr.load('../../data/dist_ks.zarr/')
p = zarr.load('../../data/p_ks.zarr/')

In [ ]:
np.testing.assert_array_equal(dask_dist,dist)
np.testing.assert_array_equal(dask_p,p)

# Conclusion

- Dask can benifit when the GPU memory is not enough;
- Dask allows multi-gpu parallel processing.